In [ ]:
https://github.com/DirkEngfer/PandasTests
Hourly NO2 measures in Bornhoeved (Schleswig-Holstein/Germany). programmer: Dirk Engfer, Germany
Provided Input data file: NO2_2019_Bornhoeved.csv

------------------------------
Original source of Input Data:
------------------------------
Quelle: Umweltbundesamt, https://www.umweltbundesamt.de/daten/luft/luftdaten/stationen
        (Abruf: 02.10.2020). Alle Uhrzeiten sind in der jeweils zum Messzeitpunkt
        gültigen Zeit (MEZ bzw. MESZ) angegeben.


In [1]:
import os, numpy as np
import pandas as pd
homedir = os.getenv('HOME')

datapath = os.path.join(homedir, 'Dokumente','python-apps','tensorflow', 'eu_air_pollution_data')
datafile = 'NO2_2019_Bornhoeved.csv'

indatapath = os.path.join(datapath,datafile)

In [ ]:
Translate the SAS SET statement and friends into Pandas:

In [2]:
df = pd.read_csv(indatapath, header=0, sep=',',usecols=[2,7,8,9])
df = df.loc[(df.Stationsname.isin(['Bornhöved', 'Kiel-Bahnhofstr. Verk.']))]
df = df.loc[(df.Messwert.str.contains(pat='-')) == False]
df['Messwert']  = df['Messwert'].astype(np.int)
from datetime import datetime
f = lambda x:datetime.strptime(x[-10:], "%d.%m.%Y")
df['mydate'] = df['Datum'].map(f)

df.sort_values(by=['Stationsname', 'mydate', 'Messwert'], ascending=[True,True, False], inplace=True, axis=0)
dailymean = df.groupby(['Stationsname', 'mydate'], sort=False, as_index=False).nth([0,1,2]).groupby(['Stationsname', 'mydate'], sort=False, as_index=False).mean()
dailymean['day_count'] = dailymean.groupby('Stationsname', sort=False, as_index=False).cumcount().add(1)
dailymean['Jahresgrenzwert'] = 40
Ki = dailymean.loc[(dailymean.Stationsname.isin(['Kiel-Bahnhofstr. Verk.']))]
Bo = dailymean.loc[(dailymean.Stationsname.isin(['Bornhöved']))]
Bo2 = Bo.copy()
Bo2['Bornh'] = 'yes'

In [ ]:
Set together 2 dataframes,
  (1) perform Append
  (2) perform Concatenation
  (3) Compare methods 1 and 2 for differences in the resulting dataframe
      i.e. Merge both results ON all columns and look at Merge-indicator variable = 'both'

In [3]:
_appended = Ki.append(Bo2, ignore_index=True, verify_integrity=False, sort=False)
_concatenated = pd.concat([Ki, Bo2], join='outer', axis=0)
compared = _appended.merge(_concatenated, on=['Stationsname', 'mydate', 'Messwert', 'day_count', 'Jahresgrenzwert', 'Bornh'], how='inner', indicator=True)
print(compared.loc[compared._merge != 'both', :])

# Some more tests on Merge facets:
d1 = pd.DataFrame({'x':['x1', 'x2', 'x2', 'x3'], 'y':['y1', 'y2a','y2b', 'y3'], 'z': ['z1', 'z2a','z2b', 'z3']})
#d1
d2 = pd.DataFrame({'x':['x2', 'x3', 'x40', 'x50'], 'y':['y10', 'y20','y20', 'y30'], 'z': ['z10', 'z20','z20', 'z30']})
d3 = d1.merge(d2, on='x', how='outer')
d4 = d1.merge(d2, on='x', how='inner')
d5 = _concatenated.copy()

Empty DataFrame
Columns: [Stationsname, mydate, Messwert, day_count, Jahresgrenzwert, Bornh, _merge]
Index: []


In [4]:
def sum(x, y, z, m):
    return (x + y + z) * m


dfw = pd.DataFrame({'A': [1, 2], 'B': [10, 20]})

dfw2 = dfw.apply(sum, args=(1, 2), m=10)
print(dfw2)

    A    B
0  40  130
1  50  230


In [32]:
df.sort_values(by=['Stationsname', 'mydate', 'Uhrzeit'], ascending=[True,True, True], inplace=True, axis=0)
df2 = df.copy()
df2.Uhrzeit.replace(regex=["'"], value='', inplace=True)
df2.Uhrzeit.replace(regex=["24:00"], value='00:00', inplace=True)
df2.Stationsname.replace(regex=["Kiel-Bahnhofstr. Verk."], value='Kiel', inplace=True)
df2['datumuhrzeit'] = df2['Datum'] + ' ' + df2['Uhrzeit']
# Example for map method and datetime operations:
f = lambda x:datetime.strptime(x[-10:], "%d.%m.%Y")
df['mydate'] = df['Datum'].map(f)
# datetime in pandas notation:
df2['mydt']   = pd.to_datetime(df2['datumuhrzeit'], format='%d.%m.%Y %H:%M')
# Subset data:
subset_df2 = df2.loc[df2.apply(lambda x: x['Uhrzeit'] in ['16:00', '17:00'] \
                      and x.Messwert < 20, axis=1)]

# Emulate SAS if-else if-else construct on multiple columns:
def set_timefor(row):
    if row["Uhrzeit"] == "16:00" and row.Datum != '31.12.2019':
        return "coffee"
    elif row.Datum == '31.12.2019':
        return "New year's eve"
    elif row["Uhrzeit"] in ["18:00", "19:00"]:
        return "dinner"
    else:
        return "other"

df2 = df2.assign(time_for=df2.apply(set_timefor, axis=1))
print(df2.info())
print(df2)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17433 entries, 0 to 17487
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Stationsname  17433 non-null  object        
 1   Datum         17433 non-null  object        
 2   Uhrzeit       17433 non-null  object        
 3   Messwert      17433 non-null  int64         
 4   mydate        17433 non-null  datetime64[ns]
 5   datumuhrzeit  17433 non-null  object        
 6   mydt          17433 non-null  datetime64[ns]
 7   time_for      17433 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 1.2+ MB
None
      Stationsname       Datum Uhrzeit  Messwert     mydate      datumuhrzeit  \
0        Bornhöved  01.01.2019   16:00         2 2019-01-01  01.01.2019 16:00   
1        Bornhöved  01.01.2019   17:00         2 2019-01-01  01.01.2019 17:00   
2        Bornhöved  01.01.2019   18:00         2 2019-01-01  01.01.2019 18: